# Problem set 2

## Team
Please write here your names and team number.

* Team name: ps-2-e
* Team members: Xiaohan Wu, Benjamin Adoba Ayida, Giulia Petrilli


## Using Colab with GitHub
To utilize GPU support for model training, we highly recommend to open this notebook with Google Colab. Simply, change the domain from 'github.com' to 'githubtocolab.com' and refresh the site to open the notebook in Colab.
If you haven't used Colab before with private repositories, make sure to grant Colab access to your private repositories (see screenshot) and after that just try to change the domain again.

Finally, you should make sure that you add a GPU to your Colab notebook. You can do so by clicking on `Runtime` →  `Change runtime type` → `Hardware accelerator`  →  `GPU`.

## Submission

Make sure that you always commit and push the changes you make in Colab back to GitHub. To do so from within a Colab notebook, click `File` → `Save a copy in GitHub`. You will be prompted to add a commit message, and after you click OK, the notebook will be pushed to your repository. Only changes that are visible in your GitHub repository on the main branch will be considered for grading. If you close Colab in your browser without pushing your changes to GitHub or saving them on Google Drive, they will be lost.

Make sure that all your work has been pushed to GitHub before the deadline.


Check that the GPU  enabled in your colab notebook by running the cell below.

In [ ]:
import torch
# Check is GPU is enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

# Get specific GPU model
if str(device) == "cuda:0":
  print("GPU: {}".format(torch.cuda.get_device_name(0)))

You will be working with the EuroSAT dataset. The dataset contains 8489 pictures of 3 different land coverage types (crop, herbaceous vegetation and river). Running the lines below will download the data and return a random picture from the dataset.

In [ ]:
from torchvision.datasets import EuroSAT
import os
import numpy as np

data = EuroSAT(root=os.getcwd(), download=True) #downloads the dataset to your current directory
print(f"The dataset has {len(data)} images")
randint = np.random.randint(len(data))

pic, tar = data[randint]
print(f"Picture number {randint} with label: {tar}")
pic

# Task 1: Transform the data (10 pt)

 Your task is to train a classifier to classify the different land usage types in the dataset.
- Implement the class `rotate` that maps pictures to flipped pictures by 90, 180, 270 or 360°. The class should return an error if you try to rotate the picture by other degrees.
- Plot a histogram with the frequencies of each class. Make sure to insert both name and label in the histogram (e.g. `AnnualCrop:0`).
- We create a class `RotateEuroSAT` that takes as input the original dataset and returns a new dataset which contains randomly rotated pictures and whose label proportion can be customized.
Implement the class method `_create_rotated_dataset` that returns this pictures using the previously implemented `rotate`.
- `RotateEuroSAT` should also take care of transforming the pictures to tensors and optionally move the tensor to a GPU device.

In [ ]:
from torch.utils.data import Subset, Dataset, random_split
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms

def rotate_picture(picture, rotation):
  '''#TODO: implemented most frequent n people'''
  # PIL uses predefined constants for transpose operations
  if rotation == 90:
    return picture.transpose(Image.ROTATE_90)
  elif rotation == 180:
    return picture.transpose(Image.ROTATE_180)
  elif rotation == 270:
    return picture.transpose(Image.ROTATE_270)
  elif rotation == 360:
    # 360 degrees rotation is no change
    return picture
  else:
    raise ValueError(f"Invalid rotation degree: {rotation}. Must be 90, 180, 270, or 360.")


def plot_histogram(data):
  # extract all labels from the dataset
  labels = [data[i][1] for i in range(len(data))]

  # Count the frequency of each label
  unique_labels, counts = np.unique(labels, return_counts=True)

  try:
    class_names = data.classes
  except AttributeError:
    class_names = [f"Class_{i}" for i in unique_labels]

  # Create labels for the histogram: Name:Label
  x_labels = [f"{class_names[i]}:{i}" for i in unique_labels]

  fig, ax = plt.subplots(figsize=(10, 6))

  # Plot the histogram
  bars = ax.bar(x_labels, counts, color='skyblue', edgecolor='black')

  # Add value labels above each bar
  for bar, count in zip(bars, counts):
      height = bar.get_height()
      ax.text(
          bar.get_x() + bar.get_width() / 2,  # x position (center of bar)
          height + max(counts) * 0.01,        # y position (slightly above bar)
          f"{int(count)}",                    # text (count)
          ha='center', va='bottom', fontsize=10, color='black'
      )

  # Add titles and labels
  ax.set_title("Class Frequency Histogram (EuroSAT)", fontsize=14, pad=15)
  ax.set_xlabel("Class Label and Name", fontsize=12)
  ax.set_ylabel("Frequency", fontsize=12)
  plt.xticks(rotation=45, ha='right') # Rotate labels for better fit
  plt.tight_layout()

  return fig, ax


new_pic = rotate_picture(pic, 90) # Example of rotating a picture by 90 degrees
same_pic = rotate_picture(pic, 360) # Example of rotating a picture by 360 degrees (should return the same picture)
fig, ax = plot_histogram(data)
fig.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

axes[0].imshow(pic)
axes[0].set_title("Original picture")
axes[1].imshow(new_pic)
axes[1].set_title("Rotated by 90°")
axes[2].imshow(same_pic)
axes[2].set_title("Rotated by 360°")

In [ ]:
class RotateEuroSAT(Dataset):
    def __init__(self,
                 original_data:Dataset,
                 length:int,
                 shares:list,
                 device=None,
                 seed=42):

        self.original_data = original_data
        self.length = length
        assert sum(shares)  == 1, "Shares must sum to 1"
        assert len(shares) == len(original_data.classes), "Shares must match number of classes"
        self.shares = shares
        self.seed = seed
        self.device = device
        self.dataset = self._create_rotated_dataset()

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        picture, label = self.dataset[idx]
        return picture, label

    def _create_rotated_dataset(self):
        """#TODO: implement solution"""
        np.random.seed(self.seed)
        rng = np.random.default_rng(self.seed)

        rotations = [90, 180, 270, 360]
        dataset = []

        # Compute number of samples per class based on shares
        samples_per_class = (np.array(self.shares) * self.length).astype(int)

        # Collect indices by class
        class_indices = {i: [] for i in range(len(self.original_data.classes))}
        for idx in range(len(self.original_data)):
            _, label = self.original_data[idx]
            class_indices[label].append(idx)

        # Build new dataset with rotated images
        for cls, n_samples in enumerate(samples_per_class):
            if n_samples == 0:
                continue

            # Randomly pick indices from this class
            chosen_indices = rng.choice(class_indices[cls], size=n_samples, replace=True)

            for i in chosen_indices:
                pic, label = self.original_data[i]

                # Apply random rotation
                rotation_angle = rng.choice(rotations)
                rotated_pic = rotate_picture(pic, rotation_angle)

                # Convert to tensor if not already
                if not isinstance(rotated_pic, torch.Tensor):
                    rotated_pic = transforms.ToTensor()(rotated_pic)

                # Move to GPU if specified
                if self.device:
                    rotated_pic = rotated_pic.to(self.device)

                dataset.append((rotated_pic, label))

        return dataset

In [ ]:
rotated_data = RotateEuroSAT(data,
                             length=10**4,
                             shares=[1 / len(data.classes) for _ in data.classes],
                             seed=42)

train_data, test_data = random_split(rotated_data, [0.8, 0.2])

## Task 2: Implement a max pooling class and a CNN model(15 pt)
Implement a classification model to predict the label of the faces in the dataset. You are free to experiment with the network architecture. However your model **must** contain:
- At least one max pooling layer, implemented with `MyMaxPool`,
- Convolutional, linear, and pooling layers only,
- At least 3 convolutional layers, with at least two different kernel sizes,
- A final output layer that is customizable to the number of classes that we want to predict.

#### Briefly explain why you chose the particular architecture you implemented (around 2-3 sentences).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyMaxPool(nn.Sequential):

  def __init__(self,
               kernel_size,
               stride=1,
               padding=0):
    super().__init__()
    self.kernel_size = kernel_size
    self.stride = stride if stride else kernel_size
    self.padding = padding

  def forward(self, x):
    # Use PyTorch functional API for pooling
    return F.max_pool2d(x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)


class MyCNNModel(nn.Sequential):

  def __init__(self, n_classes):
    super().__init__()

    # --- Convolutional layers with Batch Normalization ---
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(32)

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
    self.bn2 = nn.BatchNorm2d(64)

    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.bn3 = nn.BatchNorm2d(128)

    # --- Pooling layer ---
    self.pool = MyMaxPool(kernel_size=2, stride=2)

    # --- Fully connected layers ---
    self.fc1 = nn.Linear(128 * 8 * 8, 256)
    self.fc2 = nn.Linear(256, n_classes)

    # --- Activation function ---
    self.relu = nn.ReLU()

  def forward(self, x):
    # Block 1: Conv + BN + ReLU + Pool
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool(x)

    # Block 2: Conv + BN + ReLU + Pool
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool(x)

    # Block 3: Conv + BN + ReLU + Pool
    x = self.conv3(x)
    x = self.bn3(x)
    x = self.relu(x)
    x = self.pool(x)

    # Flatten
    x = torch.flatten(x, 1)

    # Fully connected layers
    x = self.relu(self.fc1(x))
    x = self.fc2(x)

    return x

In [ ]:
'''#TODO: print one iteration of your model to test its correctness'''
my_model = MyCNNModel(n_classes=10)
X, y = train_data[0]
my_model(X[None, :])

The CNN consists of three convolutional layers with increasing channel depth (32, 64, 128) and alternating kernel sizes (3x3 and 5x5) to capture both fine-grained and coarse spatial features in the EuroSAT images. Each convolutional layer is followed by batch normalization, ReLU and max pooling to introduce non-linearity and reduce spatial dimensions. The flattened feature maps are passed through two fully connected layers that map the extracted representations to the final class predictions.

## Training

We define a `Trainer` function to train our model that returns avg loss and avg accuracy per epoch. We set the configuration of the trainer is set in the `cfg` dictionary. Use the trainer to train your model and make sure to print and plot avg loss and accuracy using the in-built commands.

In [ ]:
from datetime import datetime as dt
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

cfg = {
    'batch_size': 64,
    'criterion': 'CrossEntropyLoss', #change to 'nn.NLLLoss' if you are applying a softmax in the last layer of your model
    'epochs': 1,
    'learning_rate': 0.001,
    'optimizer':'Adam',
    'seed':42,

}

class Trainer:

    def __init__(self, model, cfg):
        self.model = model
        self.cfg = cfg

        for key, val in cfg.items():
            setattr(self, key, val)

        self.optimizer = getattr(optim, self.optimizer)(self.model.parameters(), lr=self.learning_rate)
        self.criterion = getattr(nn, self.criterion)()


    def iter_step(self, X, Y):
        Y_pred = self.model(X)
        loss = self.criterion(Y_pred, Y)
        acc = (Y_pred.argmax(dim=-1) == Y).to(torch.float).mean()
        return loss, acc

    def train(self, dataset):
        train_dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, generator=torch.manual_seed(self.seed))
        avg_loss, avg_acc = [], []
        tot_loss, tot_acc = 0, 0
        for epoch in range(self.epochs):
            iterdata = iter(train_dataloader)
            train_size = len(iterdata)
            pbar = tqdm(iterable=range(train_size))

            for i in pbar:
                batch = next(iterdata)
                X_batch, Y_batch = batch #this is needed for compatibility with pbar
                self.model.train()
                self.optimizer.zero_grad()
                loss, acc = self.iter_step(X_batch, Y_batch)
                loss.backward()
                self.optimizer.step()
                tot_loss += loss.item()
                tot_acc += acc.item()
                avg_loss.append(tot_loss / max(1, len(avg_loss)))
                avg_acc.append(tot_acc / max(1, len(avg_acc)))
                desc = f'Epoch:{epoch} - Avg loss:{avg_loss[-1]:.5f} - Avg acc:{avg_acc[-1]:.5f}'
                pbar.set_description(desc)

        return avg_loss, avg_acc

    def test(self, dataset):
        avg_test_loss, avg_test_acc = [], []
        test_loss, test_acc = 0, 0
        self.model.eval()
        test_dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, generator=torch.manual_seed(self.seed))

        for X_batch, Y_batch in iter(test_dataloader):
            loss, acc = self.iter_step(X_batch, Y_batch)
            test_loss += loss.item()
            test_acc += acc
            avg_test_loss.append(test_loss / max(1, len(avg_test_loss)))
            avg_test_acc.append(test_acc / max(1, len(avg_test_acc)))

        return avg_test_loss, avg_test_acc

In [ ]:
'''#TODO: train your model, plot accuracy and loss by iteration (one iteration=one batch)'''
my_trainer = Trainer(my_model, cfg)
train_loss, train_acc = my_trainer.train(train_data)
fig, (ax0, ax1) = plt.subplots(1,2)
ax0.plot(range(len(train_loss)), train_loss)
ax1.plot(range(len(train_acc)), train_acc)
ax0.set_title('Training loss')
ax1.set_title('Training accuracy')

In [ ]:
'''#TODO: test your model, plot accuracy and loss by iteration (one iteration=one batch)'''
test_loss, test_acc = my_trainer.test(test_data)
fig, (ax0, ax1) = plt.subplots(1,2)
ax0.plot(range(len(test_loss)), test_loss)
ax1.plot(range(len(test_acc)), test_acc)
ax0.set_title('Test loss')
ax1.set_title('Test accuracy')

In [ ]:
# --- PERFORMANCE SUMMARY FOR MYCNNMODEL ---

print("\n=== MyCNNModel Performance Summary ===")
print(f"Final Training Loss: {train_loss[-1]:.4f}")
print(f"Final Training Accuracy: {train_acc[-1]*100:.2f}%")
print(f"Final Test Loss: {test_loss[-1]:.4f}")
print(f"Final Test Accuracy: {test_acc[-1]*100:.2f}%")

# Model architecture summary
print("\nModel Architecture:\n")
print(my_model)

## Task 3: Tune your training hyperparameters (optional, 10 pt)

Implement a method <code>grid_search</code>, which looks for the best possible learning rates and training batch sizes for your model <code>MyCNNModel</code> and returns the best possible model, the corresponding training configuration, and the final training avg losses and accuracies (as numbers).

In [ ]:
def grid_search(train_dataset, test_dataset, cfg,
                learning_rates=[10**-1, 10**-2, 10**-3],
                batch_sizes=[2**5, 2**6, 2**7]):

    best_model = None
    best_cfg = None
    best_avg_acc = 0.0
    best_avg_loss = float('inf')

    # Try all learning rates and batch sizes
    for lr in learning_rates:
        for bs in batch_sizes:
            print(f"\nTraining with learning rate={lr}, batch size={bs}")

            # Copy base configuration and update the current ones
            current_cfg = cfg.copy()
            current_cfg['learning_rate'] = lr
            current_cfg['batch_size'] = bs

            # Initialize a new model and trainer
            model = MyCNNModel(n_classes=10)
            trainer = Trainer(model, current_cfg)

            # --- TRAINING PHASE ---
            train_loss, train_acc = trainer.train(train_dataset)
            # --- TEST PHASE ---
            test_loss, test_acc = trainer.test(test_dataset)

            # Compute final averages for both train and test
            with torch.no_grad():
                final_train_loss = np.mean(train_loss[-min(10, len(train_loss)):])
                final_train_acc = np.mean(train_acc[-min(10, len(train_acc)):])
                final_test_loss = np.mean(test_loss[-min(10, len(test_loss)):])
                final_test_acc = np.mean(test_acc[-min(10, len(test_acc)):])

            print(f"  → Final Train Loss: {final_train_loss:.4f}, Train Acc: {final_train_acc:.4f}")
            print(f"  → Final Test Loss: {final_test_loss:.4f}, Test Acc: {final_test_acc:.4f}")

            # --- Selection Criterion ---
            if final_test_acc > best_avg_acc or (final_test_acc == best_avg_acc and final_test_loss < best_avg_loss):
                best_model = model
                best_cfg = current_cfg
                best_avg_loss = final_test_loss
                best_avg_acc = final_test_acc

    print("\n=== Best Configuration Found ===")
    print(f"Learning Rate: {best_cfg['learning_rate']}, Batch Size: {best_cfg['batch_size']}")
    print(f"Test Avg Loss: {best_avg_loss:.4f}, Test Avg Accuracy: {best_avg_acc:.4f}")

    return best_model, best_cfg, best_avg_loss, best_avg_acc

In [ ]:
best_model, best_cfg, best_avg_loss, best_avg_acc = grid_search(
    train_data, test_data, cfg,
    learning_rates=[10**-1, 10**-2, 10**-3],
    batch_sizes=[2**5, 2**6, 2**7]
)

print(f"\nBest model achieves Test Loss = {best_avg_loss:.2f} and Test Accuracy = {best_avg_acc:.1%}")

## Task 4: Load and fine-tune a pre-trained model (10 pt)

<ul>
  <li>Load and train a pre-trained model for classification problems, such as those made available in <a href="https://huggingface.co/docs/timm">Hugging Face's timm library</a>. </li>
  <li> Make sure to modify the output layer to be compatible with the number of classes. </li>
  <li>Print a summary of your results.</li>
  <li>Briefly explain why you chose the particular architecture you did (around 2-3 sentences).</li>
  </ul>
  
<b>Note</b>: in case you run into computing-related (e.g. memory) issues, consider choosing another model.

In [ ]:
import timm

'''#TODO: import and fine-tune a pretrained model'''
# create a small pretrained model and replace the head
loaded_model = timm.create_model(
    'resnet18', pretrained=True,
    num_classes=10,
    in_chans=3
)
loaded_trainer = Trainer(loaded_model, cfg)

In [ ]:
'''#TODO: train your model, plot accuracy and loss by iteration (one iteration=one batch)'''
train_loss, train_acc = loaded_trainer.train(train_data)
fig, (ax0, ax1) = plt.subplots(1,2)
ax0.plot(range(len(train_loss)), train_loss)
ax1.plot(range(len(train_acc)), train_acc)
ax0.set_title('Training loss')
ax1.set_title('Training accuracy')

In [ ]:
'''#TODO: test your model, plot accuracy and loss by iteration (one iteration=one batch)'''
test_loss, test_acc = loaded_trainer.test(test_data)
fig, (ax0, ax1) = plt.subplots(1,2)
ax0.plot(range(len(test_loss)), test_loss)
ax1.plot(range(len(test_acc)), test_acc)
ax0.set_title('Test loss')
ax1.set_title('Test accuracy')

In [ ]:
print("\n=== Pre-trained Summary ===")
print(f"Final Training Loss: {train_loss[-1]:.4f}")
print(f"Final Training Accuracy: {train_acc[-1]*100:.2f}%")
print(f"Final Test Loss: {test_loss[-1]:.4f}")
print(f"Final Test Accuracy: {test_acc[-1]*100:.2f}%")

#print("\nModel Architecture Summary:")
#print(timm.models.resnet.resnet18())

We fine-tuned ResNet-18, a lightweight residual convolutional neural network pre-trained on ImageNet, because it balances computational efficiency and strong generalization performance. Its residual skip connections help prevent vanishing gradients, making it effective even when fine-tuning on smaller datasets like EuroSAT. By only retraining the final layers, we leverage robust pre-learned feature extraction while adapting the model to the 10 land-cover classes.

<a name="results-and-discussion"></a>
# Task  5: Results and discussion (5pt)

Report the final metrics and make a few comments on the overall performance for the networks you implemented (3-4 lines).

| Test metric         | your model | pre-trained model | your tuned model (optional) |
|---------------------|--------------------|-------------------|-----------------------|
| Accuracy (train)           |     52.62%         |      95.63%       |       52.44%         |                     
| Loss (train)               |       1.4088        |     0.1537        |       1.4422         |    
| Accuracy (test)           |     64.82%         |      91.78%       |       69.29%         |                     
| Loss (test)               |     1.0411          |   0.3917          |        0.9798        |              
             



The results show that the pre-trained ResNet-18 significantly outperforms the custom CNN models, achieving over 91% test accuracy with a much lower loss, due to transfer learning and deep residual feature extraction.
The custom CNN performs reasonably well given its simpler architecture, reaching around 65% test accuracy, but shows signs of underfitting due to limited capacity.
After hyperparameter tuning, the CNN improves slightly to 69% test accuracy, indicating that better learning rates and batch sizes help optimize convergence but cannot match the robustness of a pre-trained model trained on large-scale ImageNet features.